<h1>Binary class image classification using CNN</h1>

In [1]:
from datasets import load_dataset
import pandas as pd
import random
import torch
from datasets import DatasetDict
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F

d:\college docs\2nd year internship\cnn\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
ds = load_dataset("autoevaluate/autoeval-staging-eval-project-sasha__dog-food-8a6c4abe-13775897")
df = pd.DataFrame(ds['train'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   image                   2100 non-null   object
 1   target                  2100 non-null   int64 
 2   evaluation_predictions  2100 non-null   object
dtypes: int64(1), object(2)
memory usage: 49.3+ KB


In [4]:
random_idx = random.randint(0, len(ds['train']) - 1)
sample = ds['train'][random_idx]
sample['image'].show()
sample['target']

1

In [ ]:
train_test_split = ds['train'].train_test_split(test_size=0.4, seed=42)

validation_test_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

split_ds = DatasetDict({
    'train': train_test_split['train'],
    'validation': validation_test_split['train'],
    'test': validation_test_split['test']
})

class bccnn(nn.Module):
	def __init__(self):
		super(bccnn, self).__init__()
		self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
		self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
		self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
		self.fc1 = nn.Linear(64 * 32 * 32, 128)  
		self.fc2 = nn.Linear(128, 1)  

	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = x.view(x.size(0), -1)  
		x = F.relu(self.fc1(x))
		x = torch.sigmoid(self.fc2(x))  
		return x

model = bccnn().to(device)

class cd(torch.utils.data.Dataset):
	def __init__(self, dataset, transform=None):
		self.transform = transform
		self.dataset = dataset

	def __len__(self):
		return len(self.dataset)

	def __getitem__(self, idx):
		image = self.dataset[idx]['image']
		target = self.dataset[idx]['target']
		if self.transform:
			image = self.transform(image)
		return image, torch.tensor(target, dtype=torch.float32)

transform = transforms.Compose([
	transforms.Resize((128, 128)),
	transforms.ToTensor()
])

train_dataset = cd(train_test_split['train'], transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

criterion = nn.BCELoss() 
optimizer = Adam(model.parameters(), lr=0.001)

model.train()
epochs = 10
for epoch in range(epochs):
	running_loss = 0.0
	for images, targets in train_loader:
		images, targets = images.to(device), targets.to(device)

		outputs = model(images)
		loss = criterion(outputs.squeeze(), targets)

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		running_loss += loss.item()

	print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/10, Loss: 0.6838386580348015
Epoch 2/10, Loss: 0.4268416464328766
Epoch 3/10, Loss: 0.32798054739832877
Epoch 4/10, Loss: 0.28277087714523075
Epoch 5/10, Loss: 0.2024968745186925
Epoch 6/10, Loss: 0.15718317190185188
Epoch 7/10, Loss: 0.13466993924230336
Epoch 8/10, Loss: 0.11810165862552821
Epoch 9/10, Loss: 0.06341828783042729
Epoch 10/10, Loss: 0.04269459063652903


In [18]:
random_test_idx = random.randint(0, len(validation_test_split['test']) - 1)

test_sample = validation_test_split['test'][random_test_idx]
test_image = test_sample['image']
test_target = test_sample['target']

test_image_tensor = transform(test_image).unsqueeze(0).to(device)

model.eval()

with torch.no_grad():
    prediction = model(test_image_tensor).item()

test_image.show()

if test_target == 1:
    print("This is a food image.")
else:
    print("This is a Dog image.")

print(f"Predicted Probability: {prediction}")
print(f"Predicted Class: {"This is a food image." if prediction >= 0.5 else "This is a Dog image."}")

This is a food image.
Predicted Probability: 0.9916180968284607
Predicted Class: This is a food image.


to test the program with our own images:

In [19]:
from PIL import Image

test_image = Image.open("download.jpg") 

test_image_tensor = transform(test_image).unsqueeze(0).to(device)

model.eval()

with torch.no_grad():
    prediction = model(test_image_tensor).item()

test_image.show()

print(f"Predicted Probability: {prediction}")
print(f"Predicted Class: {"This is a food image." if prediction >= 0.5 else "This is a Dog image."}")

Predicted Probability: 0.017795270308852196
Predicted Class: This is a Dog image.
